##### The cell below is for you to keep track of the libraries used and install those libraries quickly
##### Ensure that the proper library names are used and the syntax of `%pip install PACKAGE_NAME` is followed

In [ ]:
#%pip install pandas 
#%pip install matplotlib
# add commented pip installation lines for packages used as shown above for ease of testing
# the line should be of the format %pip install PACKAGE_NAME 

## **DO NOT CHANGE** the filepath variable
##### Instead, create a folder named 'data' in your current working directory and 
##### have the .parquet file inside that. A relative path *must* be used when loading data into pandas

In [83]:
# Can have as many cells as you want for code
import pandas as pd
import numpy as np
filepath = "./data/catB_train.parquet" 
# the initialised filepath MUST be a relative path to a folder named data that contains the parquet file

### **ALL** Code for machine learning and dataset analysis should be entered below. 
##### Ensure that your code is clear and readable.
##### Comments and Markdown notes are advised to direct attention to pieces of code you deem useful.

In [84]:
df = pd.read_parquet(filepath)

In [4]:
df.head()

,clntnum,race_desc,ctrycode_desc,clttype,stat_flag,min_occ_date,cltdob_fix,cltsex_fix,flg_substandard,flg_is_borderline_standard,...,recency_giclaim,giclaim_cnt_success,recency_giclaim_success,giclaim_cnt_unsuccess,recency_giclaim_unsuccess,flg_gi_claim_29d435_ever,flg_gi_claim_058815_ever,flg_gi_claim_42e115_ever,flg_gi_claim_856320_ever,f_purchase_lh
19550,91b546e924,Chinese,Singapore,P,ACTIVE,2017-10-31,1974-05-09,Female,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4600,896bae548c,Chinese,Singapore,P,ACTIVE,2007-05-23,1979-11-11,Male,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13337,f364439ae6,Others,Singapore,P,ACTIVE,2019-08-31,1976-01-28,Male,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15074,70f319cfe1,Chinese,Singapore,P,ACTIVE,2021-10-18,1976-03-19,Female,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19724,2647a81328,Chinese,Singapore,P,ACTIVE,2018-07-20,1995-07-31,Female,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# General Client Information

- Almost all columns have None 
- meaning of P,G,C for customer type -> personal, group and corporate? This could be correlated to e.g. household information, possibly if they purchase as a family

In [57]:
def clean_general_info(df):
    df1 = df.copy()

    # races: None in race_desc should be replaced as a value. Either unknown or others 
    df1['race_desc'] = df1['race_desc'].fillna("Others")

    # country: None should be replaced with Not Applicable 
    # Question: Need to keep both Not applicable and unknown country code? 
    df1['ctrycode_desc'] = df1['ctrycode_desc'].fillna("Not Applicable")

    # min_occ_date: calculate the duration this customer has been with singlife 
    # convert to datetime 
    df1['min_occ_date'] = pd.to_datetime(df1['min_occ_date'], errors='coerce')
    years_with_singlife = (pd.to_datetime('now') - df1['min_occ_date']).dt.days / 365.25
    mean_duration = years_with_singlife.median() # pick median instead of mean 
    df1['years_with_company'] = years_with_singlife.fillna(mean_duration).astype(int)

    # cltdob_fix: calculate age
    df1['cltdob_fix'] = pd.to_datetime(df1['cltdob_fix'], errors='coerce')
    age_in_years = (pd.to_datetime('now') - df1['cltdob_fix']).dt.days / 365.25
    mean_age = age_in_years.mean()
    df1['age'] = age_in_years.fillna(mean_age).astype(int)
    # dropping DOB column 
    df1.drop('cltdob_fix', axis=1, inplace=True)

    # None value in gender 
    df1['cltsex_fix'].unique()
    # method 1: if None then male, method2: dropna, here I used method 1
    df1['cltsex_fix'] = df1['cltsex_fix'].fillna('Male')

    return df1 



In [136]:
df1 = clean_general_info(df)
df1.head()

,clntnum,race_desc,ctrycode_desc,clttype,stat_flag,min_occ_date,cltsex_fix,flg_substandard,flg_is_borderline_standard,flg_is_revised_term,...,recency_giclaim_success,giclaim_cnt_unsuccess,recency_giclaim_unsuccess,flg_gi_claim_29d435_ever,flg_gi_claim_058815_ever,flg_gi_claim_42e115_ever,flg_gi_claim_856320_ever,f_purchase_lh,years_with_company,age
19550,91b546e924,Chinese,Singapore,P,ACTIVE,2017-10-31,Female,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,49
4600,896bae548c,Chinese,Singapore,P,ACTIVE,2007-05-23,Male,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16,44
13337,f364439ae6,Others,Singapore,P,ACTIVE,2019-08-31,Male,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,47
15074,70f319cfe1,Chinese,Singapore,P,ACTIVE,2021-10-18,Female,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,47
19724,2647a81328,Chinese,Singapore,P,ACTIVE,2018-07-20,Female,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,28


# Client Risk and Status Indicators
1. flg_substandard: Flag for substandard risk clients : client with higher than avg risk of loss 
2. flg_is_borderline_standard: Flag for borderline standard risk clients : in between standard risk and substandard risk 
3. flg_is_revised_term: Flag if customer ever has revised terms: adjustment made to the policy
4. flg_is_rental_flat: Indicates if the client lives in a rental flat : coule be related to household / useful for economic status 
5. flg_has_health_claim: Flag for clients with health insurance claims.
6. flg_has_life_claim: Flag for clients with life insurance claims.
7. flg_gi_claim: Flag for general insurance claims.
8. flg_is_proposal: Indicates if there is a policy in proposal for client: having a policy under review -> this could be quite important, as if the client is currently considering some policies, the probablity of purcashing could be higher 
9. flg_with_preauthorisation: Flag for clients with preauthorized transactions or policies: e.g. client have given permission for certain types of transcation, e.g. premium payment, 
10. flg_is_returned_mail: Flag for returned mail instances: the mail given is unresponse 

Some thoughts: 
- A few features can indicate whether the customer is actively responding(e.g. whether they already have an insurance, returned mail etc.), their economic status and their risk. Feature engineering could be along these lines 


In [137]:
df1.iloc[:,7:17].columns

Index(['flg_substandard', 'flg_is_borderline_standard', 'flg_is_revised_term',
       'flg_is_rental_flat', 'flg_has_health_claim', 'flg_has_life_claim',
       'flg_gi_claim', 'flg_is_proposal', 'flg_with_preauthorisation',
       'flg_is_returned_mail'],
      dtype='object')

In [138]:
# convert float to int 
columns_to_convert = df1.iloc[:,7:17].columns
print(columns_to_convert)

for column in columns_to_convert:
    df1[column] = df1[column].fillna(0).astype(float).astype(int)



Index(['flg_substandard', 'flg_is_borderline_standard', 'flg_is_revised_term',
       'flg_is_rental_flat', 'flg_has_health_claim', 'flg_has_life_claim',
       'flg_gi_claim', 'flg_is_proposal', 'flg_with_preauthorisation',
       'flg_is_returned_mail'],
      dtype='object')


In [139]:
# New feature 1: has health / life / general insurance -> there is no need to distinguish them as the Qn ask for general purcahase too 

# 1. Number of Insurance purchased
df1['num_total_claims'] = (df1['flg_has_health_claim'] + df1['flg_has_life_claim'] + df1['flg_gi_claim']).fillna(0)

# 2. whether they purchased insurance 
df1['flg_health_or_life_claim'] = df1['flg_has_health_claim'] | df1['flg_has_life_claim'] | df1['flg_gi_claim']

In [140]:
# 3. combine risk together: substandard risk -> level 3, boarderline -> level 2, standard -> level 1 
conditions = [
    df1['flg_substandard'] == 1,
    df1['flg_is_borderline_standard'] == 1,
    ~(df1['flg_substandard'] == 1) & ~(df1['flg_is_borderline_standard'] == 1)  # Standard risk
]

values = [3, 2, 1]

# Create a new column 'risk_level' based on the conditions, if na means standard 
df1['risk_level'] = np.select(conditions, values, default=1)

In [141]:
# 4. dropping used columns 
df1.drop(columns = ['flg_substandard','flg_is_borderline_standard','flg_has_health_claim','flg_has_life_claim','flg_gi_claim'],inplace=True)
df1.head()

,clntnum,race_desc,ctrycode_desc,clttype,stat_flag,min_occ_date,cltsex_fix,flg_is_revised_term,flg_is_rental_flat,flg_is_proposal,...,flg_gi_claim_29d435_ever,flg_gi_claim_058815_ever,flg_gi_claim_42e115_ever,flg_gi_claim_856320_ever,f_purchase_lh,years_with_company,age,num_total_claims,flg_health_or_life_claim,risk_level
19550,91b546e924,Chinese,Singapore,P,ACTIVE,2017-10-31,Female,0,0,0,...,NaN,NaN,NaN,NaN,NaN,6,49,0,0,1
4600,896bae548c,Chinese,Singapore,P,ACTIVE,2007-05-23,Male,0,0,0,...,NaN,NaN,NaN,NaN,NaN,16,44,0,0,1
13337,f364439ae6,Others,Singapore,P,ACTIVE,2019-08-31,Male,0,0,0,...,NaN,NaN,NaN,NaN,NaN,4,47,0,0,1
15074,70f319cfe1,Chinese,Singapore,P,ACTIVE,2021-10-18,Female,0,0,0,...,NaN,NaN,NaN,NaN,NaN,2,47,0,0,1
19724,2647a81328,Chinese,Singapore,P,ACTIVE,2018-07-20,Female,0,0,0,...,NaN,NaN,NaN,NaN,NaN,5,28,0,0,1


# Demographic and Household Information
- pop_20 / hh_20 = hh_size 
- hh_size_set is a rounded value of hh_size 
- There is no need to keep all, and hh_size_est will be good enough 

In [142]:
df1.iloc[:,18:28]

,is_housewife_retiree,is_sg_pr,is_class_1_2,is_dependent_in_at_least_1_policy,f_ever_declined_la,hh_20,pop_20,hh_size,hh_size_est,annual_income_est
19550,0.0,1.0,1.0,0.0,NaN,144,202,1.402778,1,C.60K-100K
4600,0.0,1.0,0.0,0.0,NaN,153,480,3.137255,3,D.30K-60K
13337,0.0,1.0,1.0,0.0,NaN,62,179,2.887097,3,A.ABOVE200K
15074,0.0,1.0,1.0,0.0,NaN,1,4,4.000000,4,B.100K-200K
19724,0.0,1.0,1.0,0.0,NaN,114,478,4.192982,>4,E.BELOW30K
...,...,...,...,...,...,...,...,...,...,...
11284,0.0,1.0,0.0,0.0,NaN,85,273,3.211765,3,E.BELOW30K
11964,0.0,1.0,0.0,0.0,NaN,0,0,0.000000,0,B.100K-200K
5390,0.0,1.0,1.0,0.0,NaN,43,145,3.372093,3,D.30K-60K
860,0.0,1.0,0.0,0.0,NaN,72,323,4.486111,>4,E.BELOW30K


In [143]:
# convert to boolean: is_housewife_retiree, is_sg_pr, is_class_1_2, is_dependent_in_at_least_1_policy 
col = df1.iloc[:,18:22].columns
for column in col:
    df1[column] = df1[column].fillna(0).astype(float).astype(int)


In [144]:
# annual income set 
df1['annual_income_est'] = df1['annual_income_est'].fillna("F.UNKNOWN")
income_mapping = {
    'F. OTHERS': 1,
    'E.BELOW30K': 2,
    'D.30K-60K': 3,
    'C.60K-100K': 4,
    'B.100K-200K': 5,
    'A.ABOVE200K': 6
}

# Apply ordinal encoding
df1['annual_income_encoded'] = df1['annual_income_est'].map(income_mapping)


In [145]:
# hh_size_est: level 0 to level 5 
df1['hh_size_est'].unique()

df1['hh_size_est'] = df1['hh_size_est'].fillna("0")
hh_size_mapping = {
    '1': 1,
    '2': 2,
    '3': 3,
    '4': 4,
    '>4': 5,
    '0': 0
}

# Apply ordinal encoding
df1['hh_size_est_encoded'] = df1['hh_size_est'].map(hh_size_mapping)

In [146]:
# dropping columns 
df1.drop(columns=['hh_20','pop_20','hh_size','hh_size_est','annual_income_est'],inplace=True)
df1.head()

,clntnum,race_desc,ctrycode_desc,clttype,stat_flag,min_occ_date,cltsex_fix,flg_is_revised_term,flg_is_rental_flat,flg_is_proposal,...,flg_gi_claim_42e115_ever,flg_gi_claim_856320_ever,f_purchase_lh,years_with_company,age,num_total_claims,flg_health_or_life_claim,risk_level,annual_income_encoded,hh_size_est_encoded
19550,91b546e924,Chinese,Singapore,P,ACTIVE,2017-10-31,Female,0,0,0,...,NaN,NaN,NaN,6,49,0,0,1,4.0,1
4600,896bae548c,Chinese,Singapore,P,ACTIVE,2007-05-23,Male,0,0,0,...,NaN,NaN,NaN,16,44,0,0,1,3.0,3
13337,f364439ae6,Others,Singapore,P,ACTIVE,2019-08-31,Male,0,0,0,...,NaN,NaN,NaN,4,47,0,0,1,6.0,3
15074,70f319cfe1,Chinese,Singapore,P,ACTIVE,2021-10-18,Female,0,0,0,...,NaN,NaN,NaN,2,47,0,0,1,5.0,4
19724,2647a81328,Chinese,Singapore,P,ACTIVE,2018-07-20,Female,0,0,0,...,NaN,NaN,NaN,5,28,0,0,1,2.0,5


# Client Consent and Communication 
- Assuming consent to call indicate higher level of activeness 
- consent to email and mail, but the email address is invalid, is equivalent to not consent to email or mail 
- use is_valid_email and is_valid_mail to change is_consent_to_mail and is_consent_to_email
- combine the consent columns into 1: level 3 if consent to call, level 1 if consent to others, sum up the consent level 

In [147]:
consent_col = df1.iloc[:,12:18].columns 
for column in consent_col:
    df1[column] = df1[column].fillna(0).astype(float).astype(int) # na can just assume they don't consent 



In [148]:
conditions = [
    (df1['is_consent_to_call'] == 1),
    (df1['is_consent_to_email'] == 1) & (df1['is_valid_email'] == 1),
    (df1['is_consent_to_mail'] == 1) & (df1['is_valid_dm'] == 1),
    (df1['is_consent_to_sms'] == 1)
]

# Define the corresponding consent levels
values = [3, 1, 1, 1]

# Create the 'consent_level' column by summing the values based on the conditions
df1['consent_level'] = np.sum(np.column_stack(conditions) * np.array(values), axis=1)


In [149]:
# sanity check 
df1.iloc[:, [*range(12, 18), -1]]

,is_consent_to_mail,is_consent_to_email,is_consent_to_call,is_consent_to_sms,is_valid_dm,is_valid_email,consent_level
19550,0,0,0,0,1,1,0
4600,0,0,0,0,1,0,0
13337,1,1,0,0,1,1,2
15074,0,0,0,0,1,1,0
19724,1,1,0,1,1,1,3
...,...,...,...,...,...,...,...
11284,0,0,0,0,0,1,0
11964,0,0,0,0,1,1,0
5390,1,1,0,0,1,1,2
860,1,1,0,0,0,1,1


In [150]:
# drop old consent columns 
df1.drop(columns=['is_consent_to_mail','is_consent_to_email','is_consent_to_call','is_consent_to_sms','is_valid_dm','is_valid_email'],inplace=True)
df1.head()

,clntnum,race_desc,ctrycode_desc,clttype,stat_flag,min_occ_date,cltsex_fix,flg_is_revised_term,flg_is_rental_flat,flg_is_proposal,...,flg_gi_claim_856320_ever,f_purchase_lh,years_with_company,age,num_total_claims,flg_health_or_life_claim,risk_level,annual_income_encoded,hh_size_est_encoded,consent_level
19550,91b546e924,Chinese,Singapore,P,ACTIVE,2017-10-31,Female,0,0,0,...,NaN,NaN,6,49,0,0,1,4.0,1,0
4600,896bae548c,Chinese,Singapore,P,ACTIVE,2007-05-23,Male,0,0,0,...,NaN,NaN,16,44,0,0,1,3.0,3,0
13337,f364439ae6,Others,Singapore,P,ACTIVE,2019-08-31,Male,0,0,0,...,NaN,NaN,4,47,0,0,1,6.0,3,2
15074,70f319cfe1,Chinese,Singapore,P,ACTIVE,2021-10-18,Female,0,0,0,...,NaN,NaN,2,47,0,0,1,5.0,4,0
19724,2647a81328,Chinese,Singapore,P,ACTIVE,2018-07-20,Female,0,0,0,...,NaN,NaN,5,28,0,0,1,2.0,5,3


# Policy and Claim History
- recency lapse: possibly referring to latest time since insurance holder failed to pay 
- recency lapse: possibly referring to latest time since insurance holder cancelled a their insurance
- total inforce policies -> policies that are still effective 
- latest being ...  -> whether the latest one they purchase has been lapsed / cancelled  

In [151]:
df1.iloc[:,16:24]

,f_ever_declined_la,n_months_last_bought_products,flg_latest_being_lapse,flg_latest_being_cancel,recency_lapse,recency_cancel,tot_inforce_pols,tot_cancel_pols
19550,NaN,1,0,0,29.0,NaN,3,NaN
4600,NaN,45,0,0,140.0,NaN,1,NaN
13337,NaN,47,0,0,NaN,NaN,1,NaN
15074,NaN,22,0,0,NaN,NaN,1,NaN
19724,NaN,62,0,0,NaN,NaN,1,NaN
...,...,...,...,...,...,...,...,...
11284,NaN,36,0,0,86.0,NaN,1,NaN
11964,NaN,407,0,0,NaN,NaN,1,NaN
5390,NaN,101,0,0,NaN,NaN,1,NaN
860,NaN,46,1,0,46.0,NaN,2,NaN


In [152]:
df1['f_ever_declined_la'].unique()
# should replace nan with 0 
df1['f_ever_declined_la'] = df1['f_ever_declined_la'].fillna(0)

In [153]:
df1[(df1['flg_latest_being_lapse'] == 1) & (df1['recency_lapse'].isna())].iloc[:,16:24]
# these 10 roles pretty annoying ... feel like dropping them 

,f_ever_declined_la,n_months_last_bought_products,flg_latest_being_lapse,flg_latest_being_cancel,recency_lapse,recency_cancel,tot_inforce_pols,tot_cancel_pols
17766,0.0,1,1,0,NaN,NaN,1,NaN
13722,1.0,12,1,0,NaN,NaN,1,NaN
5420,0.0,2,1,0,NaN,NaN,7,NaN
7570,0.0,50,1,0,NaN,NaN,2,NaN
18616,0.0,2,1,0,NaN,NaN,2,NaN
10458,0.0,2,1,0,NaN,NaN,1,NaN
8081,0.0,2,1,0,NaN,NaN,2,NaN
18628,0.0,13,1,0,NaN,NaN,1,NaN
7011,0.0,5,1,0,NaN,NaN,4,NaN
5238,0.0,2,1,0,NaN,NaN,2,NaN


In [154]:
# if nan, assume no cancellation / lapse happened, set recency_xxx to be 9999 
df1['recency_lapse'] = df1['recency_lapse'].fillna(9999)
df1['recency_cancel'] = df1['recency_cancel'].fillna(9999)

In [155]:
# total cancelled policy: if na then 0 
df1['tot_cancel_pols'] = df1['tot_cancel_pols'].fillna(0)

In [156]:
# n_months_last_bought_products
# df1['n_months_last_bought_products'].unique()
# 8 rows with negative values?? is that consider pre-paid or outlier? 

In [157]:
# Policy purchase frequency -> 
df1['avg_purchase_frequency'] = (df1['tot_cancel_pols'] + df1['tot_inforce_pols']) / df1['years_with_company']
df1['avg_cancel_rate'] = df1['tot_cancel_pols'] / (df1['tot_cancel_pols'] + df1['tot_inforce_pols']) 

In [158]:
# might want to do something to the flgs but can't think of it now 

# Anonymized Insurance Product Metrics (APE, Sum Insured, Prepaid Premiums)

In [159]:
df1.iloc[:,24:40]

,ape_gi_42e115,ape_ltc_1280bf,ape_grp_6fc3e6,ape_grp_de05ae,ape_inv_dcd836,ape_grp_945b5a,ape_grp_6a5788,ape_ltc_43b9d5,ape_grp_9cdedf,ape_lh_d0adeb,ape_grp_1581d7,ape_grp_22decf,ape_lh_507c37,ape_lh_839f8a,ape_inv_e9f316,ape_gi_a10d1b
19550,0.0,0.0,0.0,0.0,0.0,0.0,0.0,551.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4600,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13337,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15074,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19724,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,348.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11284,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11964,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5390,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
860,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0


In [160]:
# premium prepaid 
def sum_and_drop_columns(df, prefix, new_column_name):
    # Filter columns based on the specified prefix
    filtered_columns = df.filter(regex=f'^{prefix}', axis=1)
    
    # Get the column names
    sumins_columns = filtered_columns.columns
    
    # Calculate the sum along the horizontal axis and create a new column
    df[new_column_name] = filtered_columns.sum(axis=1)
    
    # Drop the original columns
    df.drop(columns=sumins_columns, inplace=True)



In [161]:
# ape 
sum_and_drop_columns(df1, 'ape_gi', 'total_ape_gi_purchased')
sum_and_drop_columns(df1, 'ape_grp', 'total_ape_grp_purchased')
sum_and_drop_columns(df1, 'ape_inv', 'total_ape_inv_purchased')
sum_and_drop_columns(df1, 'ape_lh', 'total_ape_lh_purchased')
sum_and_drop_columns(df1, 'ape_ltc', 'total_ape_ltc_purchased')
sum_and_drop_columns(df1, 'ape_', 'total_ape_others_purchased')

In [162]:
# sumins 
sum_and_drop_columns(df1, 'sumins_gi', 'total_sumins_gi_purchased')
sum_and_drop_columns(df1, 'sumins_grp', 'total_sumins_grp_purchased')
sum_and_drop_columns(df1, 'sumins_inv', 'total_sumins_inv_purchased')
sum_and_drop_columns(df1, 'sumins_lh', 'total_sumins_lh_purchased')
sum_and_drop_columns(df1, 'sumins_ltc', 'total_sumins_ltc_purchased')
sum_and_drop_columns(df1, 'sumins_', 'total_sumins_others_purchased')

In [163]:
# prempad 
sum_and_drop_columns(df1, 'prempaid_gi', 'total_prempaid_gi_purchased')
sum_and_drop_columns(df1, 'prempaid_grp', 'total_prempaid_grp_purchased')
sum_and_drop_columns(df1, 'prempaid_inv', 'total_prempaid_inv_purchased')
sum_and_drop_columns(df1, 'prempaid_lh', 'total_prempaid_lh_purchased')
sum_and_drop_columns(df1, 'prempaid_ltc', 'total_prempaid_ltc_purchased')
sum_and_drop_columns(df1, 'prempaid_', 'total_prempaid_others_purchased')

In [164]:
df1.head()

,clntnum,race_desc,ctrycode_desc,clttype,stat_flag,min_occ_date,cltsex_fix,flg_is_revised_term,flg_is_rental_flat,flg_is_proposal,...,total_sumins_inv_purchased,total_sumins_lh_purchased,total_sumins_ltc_purchased,total_sumins_others_purchased,total_prempaid_gi_purchased,total_prempaid_grp_purchased,total_prempaid_inv_purchased,total_prempaid_lh_purchased,total_prempaid_ltc_purchased,total_prempaid_others_purchased
19550,91b546e924,Chinese,Singapore,P,ACTIVE,2017-10-31,Female,0,0,0,...,0.0,0.0,1400.0,0.0,0.0,0.0,0.0,16854.0,58406.0,16854.0
4600,896bae548c,Chinese,Singapore,P,ACTIVE,2007-05-23,Male,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13337,f364439ae6,Others,Singapore,P,ACTIVE,2019-08-31,Male,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15074,70f319cfe1,Chinese,Singapore,P,ACTIVE,2021-10-18,Female,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19724,2647a81328,Chinese,Singapore,P,ACTIVE,2018-07-20,Female,0,0,0,...,0.0,400000.0,0.0,400000.0,0.0,0.0,0.0,18444.0,0.0,18444.0


# Purchase and Lapse Metrics for Specific Products
- avg-ed lapse ape 

In [170]:
# avg lapse of ape 
lapse_ape = [col for col in df1.columns if col.startswith('lapse_')]
df1['avg_lapse_ape'] = df1[lapse_ape].mean(axis=1, skipna=True)
df1.drop(columns=lapse_ape, inplace=True)

In [171]:
df1.head()

,clntnum,race_desc,ctrycode_desc,clttype,stat_flag,min_occ_date,cltsex_fix,flg_is_revised_term,flg_is_rental_flat,flg_is_proposal,...,total_sumins_lh_purchased,total_sumins_ltc_purchased,total_sumins_others_purchased,total_prempaid_gi_purchased,total_prempaid_grp_purchased,total_prempaid_inv_purchased,total_prempaid_lh_purchased,total_prempaid_ltc_purchased,total_prempaid_others_purchased,avg_lapse_ape
19550,91b546e924,Chinese,Singapore,P,ACTIVE,2017-10-31,Female,0,0,0,...,0.0,1400.0,0.0,0.0,0.0,0.0,16854.0,58406.0,16854.0,0.00
4600,896bae548c,Chinese,Singapore,P,ACTIVE,2007-05-23,Male,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.04
13337,f364439ae6,Others,Singapore,P,ACTIVE,2019-08-31,Male,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
15074,70f319cfe1,Chinese,Singapore,P,ACTIVE,2021-10-18,Female,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
19724,2647a81328,Chinese,Singapore,P,ACTIVE,2018-07-20,Female,0,0,0,...,400000.0,0.0,400000.0,0.0,0.0,0.0,18444.0,0.0,18444.0,NaN


## The cell below is **NOT** to be removed
##### The function is to be amended so that it accepts the given input (dataframe) and returns the required output (list). 
##### It is recommended to test the function out prior to submission
-------------------------------------------------------------------------------------------------------------------------------
##### The hidden_data parsed into the function below will have the same layout columns wise as the dataset *SENT* to you
##### Thus, ensure that steps taken to modify the initial dataset to fit into the model are also carried out in the function below

In [14]:
def testing_hidden_data(hidden_data: pd.DataFrame) -> list:
    '''DO NOT REMOVE THIS FUNCTION.

The function accepts a dataframe as input and return an iterable (list)
of binary classes as output.

The function should be coded to test on hidden data
and should include any preprocessing functions needed for your model to perform. 
    
All relevant code MUST be included in this function.'''
    result = [] 
    return result

##### Cell to check testing_hidden_data function

In [ ]:
# This cell should output a list of predictions.
test_df = pd.read_parquet(filepath)
test_df = test_df.drop(columns=["f_purchase_lh"])
print(testing_hidden_data(test_df))

### Please have the filename renamed and ensure that it can be run with the requirements above being met. All the best!